In [46]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
import os.path
from os import listdir
from os.path import isfile, join
import sys
import mutagen
from pprint import pprint

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

from bpm_detect import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [49]:
SAMPLE_RATE = 44100
folder = "/home/will/cv/fun/Music/"
onlyfiles = [f for f in listdir(folder) if isfile(join(folder, f)) and "mp3" in f]
print(onlyfiles)

['8.mp3', '7.mp3', '17.mp3', '24.mp3', '22.mp3', '18.mp3', '27.mp3', '12.mp3', '14.mp3', '2.mp3', '9.mp3', '28.mp3', '23.mp3', '4.mp3', '5.mp3', '15.mp3', '11.mp3', '10.mp3', '3.mp3', '6.mp3', '26.mp3', '19.mp3', '.mp3', '16.mp3', '1.mp3', '25.mp3', '13.mp3', '20.mp3', '21.mp3']


In [55]:
def pandw(s):
    with open("data.txt", "a+") as txt:
        txt.write(s+" ")

In [56]:
for file in onlyfiles:
    file = file.split(".")[0]
    print("file ", file)
    # bpm
    samps,fs = read_wav(join(folder, file+".wav"))

    data = []
    correl=[]
    bpm = 0
    n=0;
    nsamps = len(samps)
    window_samps = 10*fs        
    samps_ndx = 0;  #first sample in window_ndx 
    max_window_ndx = int(nsamps / window_samps);
    bpms = numpy.zeros(max_window_ndx)

    #iterate through all windows
    for window_ndx in range(0,max_window_ndx):

        #get a new set of samples
        #print n,":",len(bpms),":",max_window_ndx,":",fs,":",nsamps,":",samps_ndx
        data = samps[samps_ndx:samps_ndx+window_samps]
        if not ((len(data) % window_samps) == 0):
            raise(AssertionError( str(len(data) ) ) )

        bpm, correl_temp = bpm_detector(data,fs)
        if bpm == None:
            continue
        bpms[window_ndx] = bpm
        correl = correl_temp

        #iterate at the end of the loop
        samps_ndx = samps_ndx+window_samps;
        n=n+1; #counter for debug...

    bpm = numpy.median(bpms)
    print('Completed.  Estimated Beats Per Minute:', bpm)

#     n = range(0,len(correl))
#     plt.plot(n,abs(correl)); 
#     plt.show(False); #plot non-blocking
#     time.sleep(10);
#     plt.close();

    # extract info
    f = mutagen.File(join(folder, file+".mp3"))
    bitrate = f.info.bitrate / 1000
    sr = f.info.sample_rate
    CONF = {
        "bpm":bpm,
        "bps":bpm/60,
        "sr":sr,
        "x/4": 4,
    }

    offset = 3.5
    duration = None
    print(CONF)

    # down sample
    spb = sr/CONF["bps"]
    print(spb)

    nsr = 8192*bpm/60
    print(nsr)

    y, sr = librosa.load(join(folder, file+".mp3"), sr=nsr, offset=offset, duration=duration)
    y_harmonic, y_percussive = librosa.effects.hpss(y)
    chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr, hop_length=8192)
    c_hot = np.where(chromagram >= 0.9, 1, 0)

    temp = np.zeros(12)
    NOTES = ["C","Db","D","Eb","E","F","F#","G","Ab","A","Bb","B"]
    notes_per_section = []
    for i in range(1, c_hot.shape[1]):
        temp = np.add(temp, c_hot[:,i])
        if i%4 == 0:
            notes_per_section.append(temp)
            temp = np.zeros(12)

    for sec in notes_per_section:
        s = ""
        for i in range(len(sec)):
            if sec[i] == 0:
                continue
            s = s + NOTES[i]
        pandw(s)
        
    with open("data.txt", "a+") as txt:
        txt.write("\n")

file  8


/home/will/anaconda3/lib/python3.6/site-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]


Completed.  Estimated Beats Per Minute: 184.0567612687813
{'bpm': 184.0567612687813, 'bps': 3.0676126878130217, 'sr': 44100, 'x/4': 4}
14376.0
25129.883138564273
CEB
GB
EF#GB
DbEF#
EFF#
EbF#A
EbEF#GB
DF#G
DbDG
GB
DGB
DEGA
EFG
DG
CDEG
CEGB
CGB
EGB
DbEFF#
CDbE
EbABb
EbF#GAB
DEbEF#GB
CDbDF#G
G
DEGB
DEA
DEGA
DGAbAB
DF#GB
F#
F#
DA
FF#
EF#A
EbF#G
EbF#A
EbEGB
DbDEbB
CGB
DB
D
D
DEA
DB
DF#
DF#
DB
DEF#
CF#G
CEG
CDEB
DbDF#GB
GB
DB
DGB
CEB
CEB
DA
D
DF#
DEF#
DF#
DE
EF#G
CEF#G
CDEbEB
CGB
DF#G
DF#GB
DGB
CB
CB
DB
DGA
DF#G
DEF#G
DGAbA
DF#G
F#
EF#
EbF#B
EF#B
DEbF#
DbF#
G
CDE
DbDA
DFA
DGA
CDEGB
EGB
EG
GB
CEGB
CF#
EbF#A
EbF#AB
EF#G
DbDF#GAb
CG
DGB
DEGA
DEFGA
DF
CEFGB
CEG
CEB
CG
CDbEFF#B
CEFF#
EbEBbB
EbA
F#GAB
EbF#GB
CDbF#GB
GB
DG
DG
D
DF#GA
DF#A
DF#
DF#
DEG
EG
CEG
CDEB
CF#GB
DGB
DB
DB
CEB
CB
D
CDA
DF#G
DEF#B
DF#G
DG
EF#G
CB
CEB
CEF#GB
DF#GB
DB
DGB
CF#GB
CEGB
CDAB
DA
GA
D
DF#G
D
DF#GB
F#
F#B
EbF#B
EF#B
DEbFF#
CDbD
GB
DGB
D
DG
FA
DB
DFG
CDGB
EGB
EG
CDB
CEGB
CE
C
CDbD
B

file  7


/home/will/anaconda3/lib/python3.6/site-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]


Completed.  Estimated Beats Per Minute: 183.07561389676596
{'bpm': 183.07561389676596, 'bps': 3.051260231612766, 'sr': 44100, 'x/4': 4}
14453.044529961518
24995.92381737178
Eb
CEbBb
EbAb
CEb
EbAb
EbBb
EbBb
DEbBb
Eb
CEb
CEb
EbGAbBb
EbGAb
Ab
EbGAb
EbGAb
FGA
CFGAbA
FGAb
CFAb
DFG
DbFAb
FAb
FAb
CFGAb
DFF#G
DEbF
CEb
Eb
EbAb
EbAbBb
FF#GAb
EbGAb
GAb
CEbGAb
FGAb
CAbA
GAbA
FGAb
CFAb
FG
DbAb
DbFAb
CFF#GAb
CFG
G
FG
DG
DEbG
DEb
CDEb
CEbBb
DEb
DEb
DEbG
CEb
EbAb
CEbAb
CEbAb
AbBb
FBb
AbBb
EbAbBb
EbFAbBb
EbG
EbBb
AbBb
Eb
Eb
FBb
F
CDFG
CD
CDG
DG
DG
DEbGAb
Eb
EbF
DF
F
DG
DbDG
C
CGAb
Eb
Eb
DEbFBb
DF
CFBb
DEbG
DEbG
CG
CDbDG
CDG
EbAb
EbBb
CEbAb
Eb
Eb
CEbFAb
EbAb
CF#GAb
EbFF#GAb
EbGAb
CDbDF
CEFA
FGAbA
CGAb
CFGB
DbDFG
CDbF
CDbAb
GAb
CGAb
FF#G
FF#G
FG
DG
DEb
DEb
Eb
CDEbBb
DEb
DEb
DEb
CEbBb
EbFAb
EbGAb
CEbAb
AbBb
FBb
EbAbBb
EbAbBb
EbFAbBb
EbBb
EbBb
EbBb
EbGAb
Eb
EbF
CFBb
CFBb
CFGBbB
CD
CG
DbDG
DG
EbGBb
DEb
DEb
DF
DF
DFG
CDG
CDbDG
CDbG
CDGAb
DEb
Eb
CEbAb
CEb
EbAbBb
EbGAb
EbAb
GAb
CEbGAb
FG
CEA
EFA
GAb
DGAb
FGAb


...

/home/will/anaconda3/lib/python3.6/site-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]


Completed.  Estimated Beats Per Minute: 194.55882352941177
{'bpm': 194.55882352941177, 'bps': 3.2426470588235294, 'sr': 44100, 'x/4': 4}
13600.0
26563.764705882353
GA
CG
CDb
CGA
DbEA
DEA
DA
CDbDGA
GA
CDbG
CDE
CDbD
CG
G
CGA
EF#
E
DA
DA
DA
DA
EGA
DbEF#A
DbDE
DA
DA
DbD
DEAbA
EF#AbA
DbB
DbDF#B
DbF#
DGA
DA
EF#G
DEA
DEF#A
DF#A
DA
DA
EF#GA
DbEA
DbDE
DA
DbA
DbDA
DbDF#GAb
EB
CDbDF#B
DbDF#
DF#G
DGA
DF#G
EF#A
DF#AbA
DF#A
DA
DA
DbDEbEA
CDbAB
CDbB
DF#
F#
DbF#A
F#A
DbF#G
GA
DF#A
DA
F#
F#ABb
F#GAbA
EF#B
EABbB
B
AB
DE
EA
DEbEA
DF#A
DF#A
DA
DA
DbF#A
DbB
DbF#AB
DF#
DbF#A
DbF#A
DF#B
DbF#AB
DGAB
DF#B
DA
F#A
F#A
F#GAbA
EF#B
EB
B
EAB
EGA
EF#GAbA
EA
DA
DA
DA
DA
DA
DF#G
E
DA
DA
DA
DA
DGAbA
DEF#A
DbDEA
DA
DA
DbD
DbDA
DEF#GAb
EbAB
DbDF#B
DbDF#
DF#G
DGA
F#GB
EA
DF#
DA
DA
DA
DbEA
DbAB
DbF#B
DbDF#
DbF#A
DbF#B
DF#
DbF#GB
DF#GAB
DF#
DF#A
F#A
DF#GABbB
F#GAbA
EF#B
DB
B
EA
DEA
DEbE
E
EbF#
DA
DF#A
DF#A
DbDA
CDbABbB
DbEB
E
CEbE
CEbB
CGA
F#ABbB
DGB
DG
DGBb
DG
EF#
DF#
DF#A
DF#GA
F#AbA
E
EAbB
EAbB
DEbEFF#G
EFF#
DGA
DA
DB
DG

/home/will/anaconda3/lib/python3.6/site-packages/scipy/signal/signaltools.py:1344: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  out = out_full[ind]


Completed.  Estimated Beats Per Minute: 53.73968019270396
{'bpm': 53.73968019270396, 'bps': 0.895661336545066, 'sr': 44100, 'x/4': 4}
49237.36037341059
7337.25766897718


ParameterError: Filter pass-band lies beyond Nyquist